In [8]:
import cv2
import numpy as np
import dlib
import pyautogui
pyautogui.FAILSAFE = False

# 출력창 크기 설정
output_width, output_height = 1920, 1080
target_x, target_y = int(output_width/2) , int(output_height/2)

# 웹캠 지원 최대 해상도 = 640 * 480

In [9]:
# 사각형 영역에서 원을 검출하여 반환하는 함수
# params : img, rest: 튜플형 좌표 네 개의 list
def detect_circle_in_rect(img, rect):
    # 사각형 좌표를 이용해 마스크 생성
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, np.int32([rect]), (255, 255, 255))

    # 마스크 적용
    masked = cv2.bitwise_and(img, mask)
    gray = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(gray)
    blur = cv2.GaussianBlur(equalized, (5, 5), 0)
    edges = cv2.Canny(blur, 200, 200)

    # 원 검출
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=20, 
                               minRadius=5, maxRadius=20)
    
    # 원이 검출되지 않았거나 2개 이상의 원이 검출된 경우 None 반환
    if circles is None or len(circles[0]) != 1:
        return None
    
    return circles

In [10]:
# circles의 정보 입력받아 이미지에 원 둘레 & 중심점을 그리는 함수
def draw_circles(img, circles):
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(img, (x, y), r, (0, 255, 0), 1) # 원 둘레 그리기
            cv2.circle(img, (x, y), 2, (0, 0, 255), -1) # 중심점 그리기


In [11]:
# 안면의 위치 보정을 위한 보조선(타원)을 그리는 함수*
def draw_oval(ret, frame):
    # 화면 중앙 좌표 계산
    height, width = frame.shape[:2]
    center = (int(width/2), int(height/2))
    
    # 타원 그리기 : params
    axes = (100, 130) # 장축(가로축), 단축(세로축) 길이
    angle = 0 # 타원 기울기 각도
    startAngle = 0
    endAngle = 360 
    color = (0, 255, 0)
    thickness = 2
    
    #frame에 타원 그리기
    cv2.ellipse(frame, center, axes, angle, startAngle, endAngle, color, thickness)

    # 라벨링
    label = 'Face'
    label_position = (center[0] - 30, center[1] + 150) # width축, height축 params
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    font_color = (255, 255, 255)
    font_thickness = 2
    
    cv2.putText(frame, label, label_position, font, font_scale, font_color, font_thickness)
    

In [12]:
# 마우스 포인터가 이동할 좌표를 반환
def get_point_to_move(rect_coords, point_coord):
    # 사각형 내부의 포인트 좌표 추출
    point_x, point_y = point_coord[0] , point_coord[1]

    # 사각형의 좌측 상단과 우측 하단 꼭지점 좌표
    rect_x = rect_coords[0].x
    rect_y = rect_coords[0].y
    rect_width = rect_coords[1].x - rect_x
    rect_height = rect_coords[1].y - rect_y

    # 주어진 사각형 영역에서의 상대 좌표
    relative_x = point_x - rect_x
    relative_y = point_y - rect_y

    # 모니터 화면에서의 대응 좌표 계산
    target_x = int(output_width * (relative_x / rect_width))
    target_y = int(output_height * (relative_y / rect_height))

    # 좌표 반환
    return [target_x, target_y]

In [13]:
# 얼굴 인식 model, 랜드마크 인식기 생성
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./content/shape_predictor_68_face_landmarks.dat')

# 양 눈 영역을 나타내는 랜드마크 인덱스
left_eye = [36, 37, 38, 39, 40, 41]
right_eye = [42, 43, 44, 45, 46, 47]
eyes = [left_eye , right_eye]

In [14]:
# 웹캠 열기
cap = cv2.VideoCapture(0)

# 웹캠 출력 창 크기 조정 
cv2.namedWindow('Webcam', cv2.WINDOW_NORMAL)  # 창의 크기 조정을 위해 NORMAL 모드 설정
cv2.resizeWindow('Webcam', output_width, output_height)  # 원하는 크기로 조정 (lg gram16inch기준 1920*1080)

while True:
    # 프레임 읽어오기
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    
    # 안면와 캠의 거리는 고정되있다고 가정했으므로, 사용자의 안면이 위치할 영역을 고정&표시
    # 영역의 기준은 시선의 바운더리 == 모니터 바운더리(모서리)가 최대한 일치하도록 set
    # 얼굴 위치할 영역 표시
    draw_oval(ret, frame)

    # 얼굴 검출
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        # 얼굴의 68개 landmarks 검출 및 그리기
        landmarks = predictor(gray, face)
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(frame, (x, y), 3, (255, 0, 0), -1)
        cv2.circle(frame, (x, y), 3, (255, 0, 0), -1)

        # 눈 영역(사각형)의 x, y 좌표를 저장할 리스트 초기화
        left_eye_pts = []
        right_eye_pts = []

        for eye in eyes :
            mid_side_h = np.mean([landmarks.part(eye[0]).y, landmarks.part(eye[3]).y])
            up_side_dd = 2 * abs(mid_side_h - np.mean([landmarks.part(eye[1]).y, landmarks.part(eye[2]).y]))
            up_side = mid_side_h - up_side_dd
            low_side_dd = 2 * abs(mid_side_h - np.mean([landmarks.part(eye[4]).y, landmarks.part(eye[5]).y]))
            low_side = mid_side_h + low_side_dd
            mid_side_w1 = np.mean([landmarks.part(eye[1]).x, landmarks.part(eye[5]).x])
            mid_side_w2 = np.mean([landmarks.part(eye[2]).x, landmarks.part(eye[4]).x])
            left_side_dd = 0.3 * abs(mid_side_w1 - landmarks.part(eye[0]).x)
            right_side_dd = 0.3 * abs(mid_side_w2 - landmarks.part(eye[3]).x)
            left_side = landmarks.part(eye[0]).x #+ left_side_dd
            right_side = landmarks.part(eye[3]).x #- right_side_dd
            if len(left_eye_pts) == 0 :
                left_eye_pts.append((left_side, up_side)) # 좌상단
                left_eye_pts.append((right_side, up_side)) # 우상단
                left_eye_pts.append((right_side, low_side)) # 우하단
                left_eye_pts.append((left_side, low_side)) # 좌하단
            else :
                right_eye_pts.append((left_side, up_side)) # 좌상단
                right_eye_pts.append((right_side, up_side)) # 우상단
                right_eye_pts.append((right_side, low_side)) # 우하단
                right_eye_pts.append((left_side, low_side)) # 좌하단

        # 사각형 영역 그려보기
        left_eye_pts_np = np.array(left_eye_pts, np.int32)
        right_eye_pts_np = np.array(right_eye_pts, np.int32)
        cv2.polylines(frame, [left_eye_pts_np], True, (0, 255, 255), 1)
        cv2.polylines(frame, [right_eye_pts_np], True, (0, 255, 255), 1)

        # 눈동자 영역에 원 그리기
        left_circle = detect_circle_in_rect(frame, left_eye_pts)
        right_circle = detect_circle_in_rect(frame, right_eye_pts)
        draw_circles(frame, left_circle)
        draw_circles(frame, right_circle)

        # 좌표 인식 범위 2*2 배열
        left_rect = []
        right_rect = []
        left_rect.append(landmarks.part(left_eye[1]))
        left_rect.append(landmarks.part(left_eye[4]))
        #left_rect = np.array(left_rect)
        right_rect.append(landmarks.part(right_eye[1]))
        right_rect.append(landmarks.part(right_eye[4]))
        #right_rect = np.array(right_rect)

        # 두 눈동자 모두 정상적으로 검출된 경우만 마우스 포인터 이동
        # 아예 검출되지 않거나 한 쪽만 검출된 경우: 이전 프레임의 좌표 유지
        if left_circle is not None and right_circle is not None :
            left_circle = left_circle[0, 0]
            gazepoint_left = np.array(get_point_to_move(left_rect, left_circle))
            right_circle = right_circle[0, 0]
            gazepoint_right = np.array(get_point_to_move(right_rect, right_circle))
            target_x = (gazepoint_left[0] + gazepoint_right[0]) // 2
            target_y = (gazepoint_left[1] + gazepoint_right[1]) // 2
            # 마우스 이동
            pyautogui.moveTo(target_x, target_y) 


    # 화면에 프레임 출력
    cv2.imshow("Webcam", frame)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 웹캠 해제 및 창 닫기
cap.release()
cv2.destroyAllWindows()